In [67]:
# from csv import writer

# def append_list_as_row(file_name, list_of_elem):
#     # Open file in append mode
#     with open(file_name, 'a+', newline='') as write_obj:
#         # Create a writer object from csv module
#         csv_writer = writer(write_obj)
#         # Add contents of list as last row in the csv file
#         csv_writer.writerow(list_of_elem)

# for i in centers:
#     csv_path='./centers.csv'
#     append_list_as_row(csv_path, i)

In [75]:
# def append_df_row_as_row(file_name, list_of_elem):
#     # Open file in append mode
#     with open(file_name, 'a+', newline='') as write_obj:
#         # Create a writer object from csv module
#         csv_writer = writer(write_obj)
#         # Add contents of list as last row in the csv file
#         csv_writer.writerow(list_of_elem)

# for i in range(len(df)):
#     csv_path='./color_mappings.csv'
#     append_list_as_row(csv_path, [df['group'][i], df['R'][i], df['G'][i], df['B'][i]])

SyntaxError: invalid syntax (<ipython-input-72-bb9011f93708>, line 1)

In [ ]:
## START PIPE LINE

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
from sklearn.datasets import load_sample_image
from sklearn.utils import shuffle
number_of_colors = 50

faces = pd.read_json('./faces.json')

In [26]:
colors = pd.DataFrame([], columns=['R', 'G', 'B', 'imgid'])
for i in range(len(faces['imgid'])):
    r = faces['color'][i][0]
    g = faces['color'][i][1]
    b = faces['color'][i][2]
    imgid = faces['imgid'][i]
    colors.loc[i] = [r, g, b, imgid]

In [27]:
colors

,R,G,B,imgid
0,93.370066,111.507476,135.107207,001c436a-e951-4c49-aa74-aaa6854b102a
1,111.835504,147.233097,187.604939,00380494-060e-4ddb-a19e-8d78159258be
2,55.198219,78.508539,138.990552,00308711-c33d-4912-a039-c3c2e8abda85
3,76.549167,137.361032,207.137241,004ebb20-b9fc-48bd-9627-24e282f547b2
4,100.274351,109.853008,116.984683,005fbc10-1245-44a8-a977-e6b0f2ab2f2f
...,...,...,...,...
9666,195.861505,195.861505,195.861505,fffd66f0-ca02-4f62-8e89-fa64506bd87b
9667,76.218092,93.582664,133.648871,ffed9312-56d5-435d-9d93-46da8d8318ad
9668,48.723246,63.910211,96.046298,ffed7d51-3240-42f4-b419-eb92deb7a8dc
9669,80.064791,153.008938,227.195880,ffe3279a-8b4a-49aa-a4ed-305414b23c3e


In [28]:
number_of_colors = 50
estimator = KMeans(n_clusters=number_of_colors, random_state=0)

# Fit the image
estimator.fit(colors[['R', 'G', 'B']])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=50, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [29]:
df = pd.DataFrame({'group': estimator.labels_, 'R': colors['R'], 'G': colors['G'], 'B': colors['B'], 'imgid': colors['imgid']})

In [40]:
new_faces = pd.merge(faces, df[['group', 'imgid']], on='imgid', how='outer')

In [42]:
faces_json = new_faces.to_json(orient='records')


with open('faces_new.json', 'w') as f:
    f.write(faces_json)

,group,imgid
0,32,001c436a-e951-4c49-aa74-aaa6854b102a
1,28,00380494-060e-4ddb-a19e-8d78159258be
2,36,00308711-c33d-4912-a039-c3c2e8abda85
3,13,004ebb20-b9fc-48bd-9627-24e282f547b2
4,22,005fbc10-1245-44a8-a977-e6b0f2ab2f2f
...,...,...
9666,9,fffd66f0-ca02-4f62-8e89-fa64506bd87b
9667,25,ffed9312-56d5-435d-9d93-46da8d8318ad
9668,26,ffed7d51-3240-42f4-b419-eb92deb7a8dc
9669,30,ffe3279a-8b4a-49aa-a4ed-305414b23c3e


In [58]:
## Get centers of groups

centers = estimator.cluster_centers_.round()
centers

center_df = pd.DataFrame([], columns=['R', 'G', 'B', 'group_id'])

for i in range(len(centers)):
    r = centers[i][0]
    g = centers[i][1]
    b = centers[i][2]
    center_df.loc[i] = [r,g,b, int(i)]

In [60]:
center_df

centers_json = center_df.to_json(orient='records')


with open('group_centers.json', 'w') as f:
    f.write(centers_json)

In [77]:
dfGrouped = new_faces.groupby(['gender', 'age', 'group'])
# dfGrouped.groups
# dfGrouped.agg('count')
# dfGrouped.agg(np.size)['imgid']

In [78]:
dfGrouped.agg('count').reset_index()

,gender,age,group,imgid,faceid,box,points,color
0,Female,(0-2),0,23,23,23,23,23
1,Female,(0-2),1,11,11,11,11,11
2,Female,(0-2),2,8,8,8,8,8
3,Female,(0-2),3,9,9,9,9,9
4,Female,(0-2),5,1,1,1,1,1
...,...,...,...,...,...,...,...,...
739,Male,(8-12),45,19,19,19,19,19
740,Male,(8-12),46,11,11,11,11,11
741,Male,(8-12),47,9,9,9,9,9
742,Male,(8-12),48,14,14,14,14,14
